# Chapter 4

In [9]:
from sqlalchemy import create_engine, MetaData
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

In [10]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255)),
             Column('count', Integer()),
             Column('amount', Float()),
             Column('valid', Boolean())
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print table details
print(repr(data))


Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>), schema=None)


In [18]:
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

In [19]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255), unique=True),
             Column('count', Integer(), default=1),
             Column('amount', Float()),
             Column('valid', Boolean(), default=False)
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print the table details
print(repr(metadata.tables['data']))


Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>, default=ColumnDefault(1)), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>, default=ColumnDefault(False)), schema=None)


In [20]:
# data prep
connection = engine

In [16]:
# Import insert and select from sqlalchemy
from sqlalchemy import insert, select

# Build an insert statement to insert a record into the data table: stmt
stmt = insert(data).values(name='Anna', count=1, amount=1000.0, valid=True)

# Execute the statement via the connection: results
results = connection.execute(stmt)

# Print result rowcount
print(results.rowcount)

# Build a select statement to validate the insert
stmt = select([data]).where(data.columns.name == 'Anna')

# Print the result of executing the query.
print(connection.execute(stmt).first())


1
('Anna', 1, 1000.0, True)


In [22]:
# data prep
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255), unique=True),
             Column('count', Integer(), default=1),
             Column('amount', Float()),
             Column('valid', Boolean(), default=False)
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print the table details
print(repr(metadata.tables['data']))

connection = engine

Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>, default=ColumnDefault(1)), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>, default=ColumnDefault(False)), schema=None)


In [23]:
# Build a list of dictionaries: values_list
values_list = [
    {'name': 'Anna', 'count': 1, 'amount': 1000.0, 'valid': True},
    {'name': 'Taylor', 'count': 1, 'amount': 750.0, 'valid': False}
]

# Build an insert statement for the data table: stmt
stmt = insert(data)

# Execute stmt with the values_list: results
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)


2


In [32]:
# data prep
engine = create_engine("sqlite:///census_extra.sqlite")
connection = engine.connect()
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)

import csv
csvfile = open('census.csv')
csv_reader = csv.reader(csvfile)

In [33]:
# Create a insert statement for census: stmt
stmt = insert(census)

# Create an empty list and zeroed row count: values_list, total_rowcount
values_list = []
total_rowcount = 0

# Enumerate the rows of csv_reader
for idx, row in enumerate(csv_reader):
    #create data and append to values_list
    data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3],
            'pop2008': row[4]}
    values_list.append(data)

    # Check to see if divisible by 51
    if idx % 51 == 0:
        results = connection.execute(stmt, values_list)
        total_rowcount += results.rowcount
        values_list = []

# Print total rowcount
print(total_rowcount)


8722


In [34]:
# data prep
engine = create_engine("sqlite:///census_extra.sqlite")
connection = engine.connect()
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)
more_results = True
state_count = {}

results_proxy = connection.execute("select state from census")

In [35]:
# Start a while loop checking for more results
while more_results:
    # Fetch the first 50 results from the ResultProxy: partial_results
    partial_results = results_proxy.fetchmany(50)

    # if empty list, set more_results to False
    if partial_results == []:
        more_results = False

    # Loop over the fetched records and increment the count for the state
    for row in partial_results:
        if row.state in state_count:
            state_count[row.state] += 1
        else:
             state_count[row.state] = 1

# Close the ResultProxy, and thus the connection
results_proxy.close()

# Print the count by state
print(state_count)


{'Illinois': 344, 'New Jersey': 344, 'District of Columbia': 344, 'North Dakota': 344, 'Florida': 344, 'Maryland': 344, 'Idaho': 344, 'Massachusetts': 344, 'Oregon': 344, 'Nevada': 344, 'Michigan': 344, 'Wisconsin': 344, 'Missouri': 344, 'Washington': 344, 'North Carolina': 344, 'Arizona': 344, 'Arkansas': 344, 'Colorado': 344, 'Indiana': 344, 'Pennsylvania': 344, 'Hawaii': 344, 'Kansas': 344, 'Louisiana': 344, 'Alabama': 344, 'Minnesota': 344, 'South Dakota': 344, 'New York': 344, 'California': 344, 'Connecticut': 344, 'Ohio': 344, 'Rhode Island': 344, 'Georgia': 344, 'South Carolina': 344, 'Alaska': 344, 'Delaware': 344, 'Tennessee': 344, 'Vermont': 344, 'Montana': 344, 'Kentucky': 344, 'Utah': 344, 'Nebraska': 344, 'West Virginia': 344, 'Iowa': 344, 'Wyoming': 344, 'Maine': 344, 'New Hampshire': 344, 'Mississippi': 344, 'Oklahoma': 344, 'New Mexico': 344, 'Virginia': 344, 'Texas': 294}


In [36]:
# data prep
from sqlalchemy import update
engine = create_engine('sqlite:///census_extra.sqlite')
metadata = MetaData()
state_fact = Table('state_fact', metadata, autoload=True, autoload_with=engine)
connection = engine

In [37]:
# Build a select statement: select_stmt
select_stmt = select([state_fact]).where(state_fact.columns.name == 'New York')

# Print the results of executing the select_stmt
print(connection.execute(select_stmt).fetchall())

# Build a statement to update the fips_state to 36: stmt
stmt = update(state_fact).values(fips_state = 36)

# Append a where clause to limit it to records for New York state
stmt = stmt.where(state_fact.columns.name == 'New York')

# Execute the statement: results
results = connection.execute(stmt)

# Print rowcount
print(results.rowcount)

# Execute the select_stmt again to view the changes
print(connection.execute(select_stmt).fetchall())


[('32', 'New York', 'NY', 'USA', 'state', '10', 'current', 'occupied', '', '36', 'N.Y.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '2')]
1
[('32', 'New York', 'NY', 'USA', 'state', '10', 'current', 'occupied', '', '36', 'N.Y.', 'II', '1', 'Northeast', '2', 'Mid-Atlantic', '2')]


In [65]:
# data prep - create table, and input from csv having dumped from datacamp... phew
engine = create_engine("sqlite:///census_extra.sqlite")
connection = engine.connect()
metadata = MetaData()
state_fact = Table('state_fact', metadata, autoload=True, autoload_with=engine)
metadata = MetaData()

# create table
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

# Define a new table with a name, count, amount, and valid column: data
flat_census = Table('flat_census', metadata,
             Column('state_name', String(256)),
             Column('fips_code', String(256))
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print table details
print(repr(flat_census))

# create csv file - this wasted an hour+
# csv.reader leaves single quotes alone
# so the db values INCLUDE the single quote, and never equal 
# the state_fact columns
# just remove all quotes, and let the DB column type alter
# the storage value

# open csv file ready to insert
import csv
file = open('flat_census.csv')
csv_reader = csv.reader(file)

# Create a insert statement for census: stmt
stmt = insert(flat_census)

# Create an empty list and zeroed row count: values_list, total_rowcount
values_list = []
total_rowcount = 0

# Enumerate the rows of csv_reader
for idx, row in enumerate(csv_reader):
    #create data and append to values_list
    data = {'state_name': row[0], 'fips_code': row[1]}
    values_list.append(data)

    # Check to see if divisible by 51
    if idx % 10 == 0:
        results = connection.execute(stmt, values_list)
        total_rowcount += results.rowcount
        values_list = []

# Print total rowcount
print(total_rowcount)


Table('flat_census', MetaData(bind=None), Column('state_name', String(length=256), table=<flat_census>), Column('fips_code', String(length=256), table=<flat_census>), schema=None)
51


In [66]:
# Build a statement to select name from state_fact: stmt
fips_stmt = select([state_fact.columns.name])

# Append a where clause to Match the fips_state to flat_census fips_code
fips_stmt = fips_stmt.where(
    state_fact.columns.fips_state == flat_census.columns.fips_code)

# Build an update statement to set the name to fips_stmt: update_stmt
update_stmt = update(flat_census).values(state_name=fips_stmt)

# Execute update_stmt: results
results = connection.execute(update_stmt)

# Print rowcount
print(results.rowcount)


51


In [67]:
res = connection.execute('select * from flat_census')
for r in res:
    print(r)

('Illinois', '17')
('New Jersey', '34')
('North Dakota', '38')
('Oregon', '41')
('Washington DC', '11')
('Wisconsin', '55')
('Arizona', '4')
('Arkansas', '5')
('Colorado', '8')
('Hawaii', '15')
('Kansas', '20')
('Louisiana', '22')
('Montana', '30')
('Nebraska', '31')
('Oklahoma', '40')
('Idaho', '16')
('Massachusetts', '25')
('Michigan', '26')
('Missouri', '29')
('North Carolina', '37')
('Ohio', '39')
('Rhode Island', '44')
('South Carolina', '45')
('Wyoming', '56')
('Indiana', '18')
('Pennsylvania', '42')
('South Dakota', '46')
('Tennessee', '47')
('Vermont', '50')
('Alaska', '2')
('Delaware', '10')
('Kentucky', '21')
('Mississippi', '28')
('Virginia', '51')
('Florida', '12')
('Maryland', '24')
('Nevada', '32')
('Washington', '53')
('California', '6')
('Connecticut', '9')
('Georgia', '13')
('Iowa', '19')
('Maine', '23')
('New Hampshire', '33')
('New Mexico', '35')
('Texas', '48')
('Alabama', '1')
('Minnesota', '27')
('New York', '36')
('Utah', '49')
('West Virginia', '54')


#### data prep
Copy a census db

```
cp census.sqlite census_delete.sqlite
```

In [68]:
engine = create_engine("sqlite:///census_delete.sqlite")
connection = engine.connect()
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)

In [69]:
# Import delete, select
from sqlalchemy import delete, select

# Build a statement to empty the census table: stmt
stmt = delete(census)

# Execute the statement: results
results = connection.execute(stmt)

# Print affected rowcount
print(results.rowcount)

# Build a statement to select all records from the census table
stmt = select([census])

# Print the results of executing the statement to verify there are no rows
print(connection.execute(stmt).fetchall())


8772
[]


In [74]:
# data prep
# cp census.sqlite census_delete.sqlite
from sqlalchemy import func, and_
engine = create_engine("sqlite:///census_delete.sqlite")
connection = engine.connect()
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)

In [75]:
# Build a statement to count records using the sex column for Men ('M') age 36: stmt
stmt = select([func.count(census.columns.sex)]).where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the select statement and use the scalar() fetch method to save the record count
to_delete = connection.execute(stmt).scalar()

# Build a statement to delete records from the census table: stmt_del
stmt_del = delete(census)

# Append a where clause to target Men ('M') age 36
stmt_del = stmt_del.where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the statement: results
results = connection.execute(stmt_del)

# Print affected rowcount and to_delete record count, make sure they match
print(results.rowcount, to_delete)


51 51


In [76]:
# data prep
engine = create_engine("sqlite:///census_delete.sqlite")
connection = engine.connect()
metadata = MetaData()
state_fact = Table('state_fact', metadata, autoload=True, autoload_with=engine)
census = Table('census', metadata, autoload=True, autoload_with=engine)

In [78]:
# Drop the state_fact table
state_fact.drop(engine)

# Check to see if state_fact exists
print(state_fact.exists(engine))

# Drop all tables
metadata.drop_all(engine)

# Check to see if census exists
print(census.exists(engine))


False
False
